In [4]:
from src.service.HwpService import HwpService

service = HwpService()
filePath = '/Users/dobby/Downloads/7월 계측보고서(신안산선2공구 환기구#3).hwp'

xml = service.hwp2xml(filePath)
columnTag, textTag = service.findTag(xml, '일자별 계측 현황')
xmlData = service.setTableData(columnTag, textTag)

xmlData

unmatched field end


[{'table-id': 98,
  'row': 0,
  'rowspan': 2,
  'col': 0,
  'colspan': 1,
  'text': '일 자'},
 {'table-id': 98,
  'row': 0,
  'rowspan': 2,
  'col': 1,
  'colspan': 1,
  'text': '계측위치'},
 {'table-id': 98,
  'row': 0,
  'rowspan': 2,
  'col': 2,
  'colspan': 1,
  'text': '발파횟수'},
 {'table-id': 98,
  'row': 0,
  'rowspan': 2,
  'col': 3,
  'colspan': 1,
  'text': '시간'},
 {'table-id': 98,
  'row': 0,
  'rowspan': 1,
  'col': 4,
  'colspan': 3,
  'text': '관리기준'},
 {'table-id': 98,
  'row': 0,
  'rowspan': 1,
  'col': 7,
  'colspan': 3,
  'text': '측정결과'},
 {'table-id': 98,
  'row': 1,
  'rowspan': 1,
  'col': 4,
  'colspan': 1,
  'text': '진동속도(cm/s)'},
 {'table-id': 98,
  'row': 1,
  'rowspan': 1,
  'col': 5,
  'colspan': 1,
  'text': '진동레벨(dB(V))'},
 {'table-id': 98,
  'row': 1,
  'rowspan': 1,
  'col': 6,
  'colspan': 1,
  'text': '소음레벨(dB(A))'},
 {'table-id': 98,
  'row': 1,
  'rowspan': 1,
  'col': 7,
  'colspan': 1,
  'text': '진동속도(cm/s)'},
 {'table-id': 98,
  'row': 1,
  'rowspan': 1,
 

In [9]:
tableId = sorted(set([i['table-id'] for i in xmlData]))
xmlData = [[i for i in xmlData if i['table-id'] == id] for id in tableId]

xmlData

TypeError: list indices must be integers or slices, not str

In [10]:
TARGET_TEXT = [ '일자', '진동속도', '진동레벨', '소음','cm/s', 'dB(V)', 'dB(A)']

filteredData = [
    sublist for sublist in xmlData
    if any( entry['row'] in [0, 1] and any(keyword in entry['text'] for keyword in TARGET_TEXT) for entry in sublist )
]

filteredData


[[{'table-id': 98,
   'row': 0,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '일 자'},
  {'table-id': 98,
   'row': 0,
   'rowspan': 2,
   'col': 1,
   'colspan': 1,
   'text': '계측위치'},
  {'table-id': 98,
   'row': 0,
   'rowspan': 2,
   'col': 2,
   'colspan': 1,
   'text': '발파횟수'},
  {'table-id': 98,
   'row': 0,
   'rowspan': 2,
   'col': 3,
   'colspan': 1,
   'text': '시간'},
  {'table-id': 98,
   'row': 0,
   'rowspan': 1,
   'col': 4,
   'colspan': 3,
   'text': '관리기준'},
  {'table-id': 98,
   'row': 0,
   'rowspan': 1,
   'col': 7,
   'colspan': 3,
   'text': '측정결과'},
  {'table-id': 98,
   'row': 1,
   'rowspan': 1,
   'col': 4,
   'colspan': 1,
   'text': '진동속도(cm/s)'},
  {'table-id': 98,
   'row': 1,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': '진동레벨(dB(V))'},
  {'table-id': 98,
   'row': 1,
   'rowspan': 1,
   'col': 6,
   'colspan': 1,
   'text': '소음레벨(dB(A))'},
  {'table-id': 98,
   'row': 1,
   'rowspan': 1,
   'col': 7,
   'colspan': 1,
   'text': '진

In [11]:
import copy, ast
from collections import defaultdict, Counter
from itertools import cycle, chain
from src.dto.CustomDefaultDict import CustomDefaultDict
        
def expandData(data: dict):
    expandData = []

    for i in range(data['colspan']):
        tempData = copy.deepcopy(data)
        tempData['colspan'] = 1
        tempData['col'] += i

        expandData.append(tempData)

    return expandData

def createRange(dataList: list):
    mergeDataList = [mergeData for mergeData in dataList if mergeData['rowspan'] > 1]
    
    mergeRanges = CustomDefaultDict()
    for data in mergeDataList:
        row = data['row']
        mergeRanges.append(f'{data}', list(range(row + 1, row + data['rowspan'])))
    
    return dict(mergeRanges)
    

def test2(dictData: list[dict]):
    data = list(chain.from_iterable(expandData(d) for d in dictData if d['row'] not in [0, 1]))
    columns = [i for i in dictData if i['row'] in [0, 1] and i['colspan'] < 2]

    mergeCellRanges = createRange(data)

    for i in mergeCellRanges:
        for j in mergeCellRanges[i]:
            bjbj = copy.deepcopy(ast.literal_eval(i))
            bjbj['row'] = j
            data.append(bjbj)

    result = []
    sortedData = sorted(data, key=lambda x: (x['row'], x['col']))
    for row in list(set([row['row'] for row in data])):
        rd = CustomDefaultDict()
        for datum, column in zip([data for data in sortedData if data['row'] == row], cycle(columns)):
            rd.append(column['text'], datum['text'])
        result.append(dict(rd))

    
    for zx in result:
        print(dict(zx))

    return result
    

zz = test2([dictData for dictData in filteredData[0] if dictData['text']])

    


{'일 자': '7월1일', '계측위치': '수풀안길 19-30', '발파횟수': '1회', '시간': '11:20', '진동속도(cm/s)': ['0.2', '0.007'], '진동레벨(dB(V))': ['75', '48.33'], '소음레벨(dB(A))': ['75', '36.79']}
{'일 자': '7월1일', '계측위치': '수풀안길 19-30', '발파횟수': '2회', '시간': '14:59', '진동속도(cm/s)': ['0.2', '0.038'], '진동레벨(dB(V))': ['75', '51.79'], '소음레벨(dB(A))': ['75', '44.27']}
{'일 자': '7월1일', '계측위치': '안산벌말길 95', '발파횟수': '3회', '시간': '15:52', '진동속도(cm/s)': ['1.0', '0.071'], '진동레벨(dB(V))': ['-', '60.02'], '소음레벨(dB(A))': ['-', '39.19']}
{'일 자': '7월1일', '계측위치': '수풀안길 19-30', '발파횟수': '4회', '시간': '19:39', '진동속도(cm/s)': ['0.2', '0.026'], '진동레벨(dB(V))': ['75', '57.33'], '소음레벨(dB(A))': ['75', '32.35']}
{'일 자': '7월2일', '계측위치': '수풀안길 19-30', '발파횟수': '1회', '시간': '7:05', '진동속도(cm/s)': ['0.2', '0.014'], '진동레벨(dB(V))': ['75', '47.41'], '소음레벨(dB(A))': ['75', '34.54']}
{'일 자': '7월2일', '계측위치': '안산벌말길 95', '발파횟수': '2회', '시간': '10:29', '진동속도(cm/s)': ['1.0', '0.06'], '진동레벨(dB(V))': ['-', '58.41'], '소음레벨(dB(A))': ['-', '43.33']}
{'일 자': '7월2일', '계측위치': '수풀안길 19

In [12]:
def tee(a, b):
    print(a, b)

a = {'a': 3, 'b': 4}

print(tee(**a))

3 4
None


In [13]:
import numpy as np

np.nan < 50

False